# Ontario Section Pilot - Crosswalk by Level 2 Sections

**Approach:** Process PDF by Level 2 sections (from reference CSV) for coherent extraction.

**Ontario-Specific Features:**
- **H/P Columns**: Hospital vs Professional/Office setting → separate rows
- **Surgical Columns**: Surg/Asst/Anae fees → separate rows per fee type
- **Level 3**: Extracted by LLM (e.g., "INCISION", "EXCISION")

**Output Columns:**
- `Setting`: Hospital, Professional, N/A
- `Fee_Type`: Surgeon, Assistant, Anaesthesia, Standard

## Cell 1: Setup

In [ ]:
!pip install openai pandas pdfplumber openpyxl tqdm PyMuPDF -q
print('Ready')

## Cell 2: Upload Ontario PDF

In [ ]:
from google.colab import files

print("Upload Ontario Schedule of Benefits PDF:")
print("(ON - February 20, 2024 (effective April 1, 2024).pdf)")
uploaded = files.upload()

ON_PDF = None
for f in uploaded.keys():
    ON_PDF = f
    break

if ON_PDF:
    print(f"\nLoaded: {ON_PDF}")
else:
    print("ERROR: No file uploaded")

## Cell 3: Upload Section Reference CSV

In [ ]:
import pandas as pd
from google.colab import files

print("Upload on_section_reference_full.csv:")
uploaded_ref = files.upload()

section_ref_file = list(uploaded_ref.keys())[0]
df_section_ref = pd.read_csv(section_ref_file)

# Sort by page_start
df_section_ref = df_section_ref.sort_values('page_start').reset_index(drop=True)

# Fill empty level_2 with level_1 value
df_section_ref['level_2'] = df_section_ref['level_2'].fillna('')

print(f"\nLoaded {len(df_section_ref)} section entries")
print(f"Unique Level 1 sections: {df_section_ref['level_1'].nunique()}")
print(f"\nLevel 1 sections:")
for l1 in df_section_ref['level_1'].unique():
    count = len(df_section_ref[df_section_ref['level_1'] == l1])
    print(f"  {l1}: {count} subsections")

## Cell 4: API Key

In [ ]:
OPENAI_API_KEY = ""  # <-- Paste your key here

if not OPENAI_API_KEY:
    from getpass import getpass
    OPENAI_API_KEY = getpass("API Key: ")

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)
print("API ready")

## Cell 5: Split PDF into Level 2 Section Chunks

Extract text for each Level 2 section based on page ranges from CSV.
Skip General Preamble (pages 1-126) and Appendices.

In [ ]:
import pdfplumber
from tqdm.notebook import tqdm

print("Loading PDF and splitting into Level 2 sections...")
print("="*70)

# First, load all pages
pdf_pages = {}
with pdfplumber.open(ON_PDF) as pdf:
    total_pages = len(pdf.pages)
    for i, page in enumerate(tqdm(pdf.pages, desc="Loading pages")):
        page_num = i + 1
        try:
            text = page.extract_text()
            if text:
                pdf_pages[page_num] = text
        except:
            pass

print(f"\nLoaded {len(pdf_pages)} pages from PDF (total: {total_pages})")

# Sections to skip
SKIP_LEVEL1 = [
    "General Preamble",
    "Appendix A",
    "Appendix B",
    "Appendix C",
    "Appendix D",
    "Appendix F",
    "Appendix G",
    "Appendix H",
    "Appendix J",
    "Appendix Q",
    "Numeric Index",
]

# Build section chunks using Level 2 page ranges
section_chunks = {}

for idx, row in df_section_ref.iterrows():
    level_1 = row['level_1']
    level_2 = row['level_2'] if row['level_2'] else level_1
    start_page = int(row['page_start'])
    
    # Skip preamble and appendices
    if level_1 in SKIP_LEVEL1:
        continue
    
    # Create unique section key
    section_key = f"{level_1} | {level_2}" if level_2 != level_1 else level_1
    
    # End page is start of next section - 1, or last page of PDF
    if idx + 1 < len(df_section_ref):
        end_page = int(df_section_ref.iloc[idx + 1]['page_start']) - 1
    else:
        end_page = total_pages
    
    # Extract text for this section
    section_text = ""
    pages_in_section = []
    for pg in range(start_page, end_page + 1):
        if pg in pdf_pages:
            section_text += f"\n=== PAGE {pg} ===\n{pdf_pages[pg]}"
            pages_in_section.append(pg)
    
    section_chunks[section_key] = {
        'text': section_text,
        'level_1': level_1,
        'level_2': level_2,
        'start_page': start_page,
        'end_page': end_page,
        'page_count': len(pages_in_section),
        'char_count': len(section_text)
    }

print(f"\nCreated {len(section_chunks)} section chunks (after skipping preamble/appendices):")
print("-"*70)
for name, info in section_chunks.items():
    print(f"  {name[:55]:55} | Pages {info['start_page']:3}-{info['end_page']:3} ({info['page_count']:2} pgs) | {info['char_count']:,} chars")

print("\nSection chunks ready for processing")

## Cell 6: Alberta Code Definition + Prompt Builder

In [ ]:
import json
import re

# Alberta code definition
AB_CODE = "03.03CV"
AB_DESC = "Telehealth consultation"
AB_FEE = 25.09

AB_CLINICAL_DEFINITION = """Assessment of a patient's condition via telephone or secure videoconference.

NOTE:
- At minimum: limited assessment requiring history related to presenting problems, appropriate records review, and advice to the patient
- Total physician time spent providing patient care must be MINIMUM 10 MINUTES
- If less than 10 minutes same day, must use HSC 03.01AD instead
- May only be claimed if service was initiated by the patient or their agent
- May only be claimed if service is personally rendered by the physician
- Benefit includes ordering appropriate diagnostic tests and discussion with patient
- Patient record must include detailed summary of all services including start/stop times
- Time spent on administrative tasks cannot be claimed
- May NOT be claimed same day as: 03.01AD, 03.01S, 03.01T, 03.03FV, 03.05JR, 03.08CV, 08.19CV, 08.19CW, or 08.19CX by same physician for same patient
- May NOT be claimed same day as in-person visit or consultation by same physician for same patient

Category: V Visit (Virtual)
Base rate: $25.09"""

# Tracking
total_cost = 0.0
total_calls = 0

def track_cost(inp, out):
    global total_cost, total_calls
    total_cost += (inp/1e6)*3.0 + (out/1e6)*15.0
    total_calls += 1

def build_section_prompt(section_key, section_info):
    """Build prompt for processing a complete Level 2 section."""
    level_1 = section_info['level_1']
    level_2 = section_info['level_2']
    section_text = section_info['text']
    start_page = section_info['start_page']
    end_page = section_info['end_page']
    
    return f"""You are a senior physician billing specialist mapping Alberta fee codes to Ontario equivalents.

ALBERTA CODE TO MATCH:
- Code: {AB_CODE}
- Description: {AB_DESC}
- Fee: ${AB_FEE}

CLINICAL SERVICE DEFINITION:
{AB_CLINICAL_DEFINITION}

This is a BASIC PATIENT-FACING virtual visit by any physician (not specialist-specific, not physician-to-physician).

You are reviewing the section: {level_1} > {level_2}
Pages {start_page} to {end_page}

ONTARIO SCHEDULE OF BENEFITS - SECTION:

{section_text}

TASK:
Find ALL Ontario codes in this section that bill for patient-facing virtual assessments (telephone or video consultations with patients).

ONTARIO-SPECIFIC EXTRACTION RULES:

1. **H/P COLUMNS (Setting)**:
   - If a code has BOTH H (Hospital) and P (Professional/Office) fees, create SEPARATE entries for each
   - H = Hospital setting, P = Professional/Office setting
   - If only one fee exists, use that setting

2. **SURGICAL FEE COLUMNS**:
   - Surg = Surgeon fee → create entry with fee_type "Surgeon"
   - Asst = Assistant fee → create entry with fee_type "Assistant" (skip if "nil")
   - Anae = Anaesthesia units → create entry with fee_type "Anaesthesia" (these are TIME UNITS, not dollars)

3. **CODE PREFIXES** (indicate service type, NOT setting):
   - A = Assessments/consultations
   - E = Diagnostic/therapeutic procedures
   - G = General listings
   - K = Special visit premiums
   - Z = Surgical procedures

ACCURACY RULES:

1. **ONLY REAL CODES**: Return ONLY codes that LITERALLY appear in the text above.
   - Copy the EXACT code as shown (e.g., A003, K017, Z101)
   - NEVER invent, fabricate, or guess codes

2. **EXACT VALUES**: Copy fee EXACTLY as shown in the document
   - Use exact decimal values (e.g., "87.35" not "87.00")
   - For Anae column, these are UNITS not dollars

3. **FULL DESCRIPTIONS - CLIENT READY FORMAT**:
   - Copy the COMPLETE service description as written in the schedule
   - Do NOT abbreviate or truncate
   - Use sentence case for consistency
   - Include qualifying details (e.g., "minimum 50 minutes")

4. **LEVEL 3 EXTRACTION**:
   - Extract the subsection heading the code appears under (e.g., "INCISION", "EXCISION", "GENERAL LISTINGS")
   - This becomes level_3_heading

5. **MODALITY**: Only include modalities explicitly stated
   - "telephone" = text says telephone/phone
   - "video" = text says video/videoconference
   - "both" = text explicitly allows BOTH, or doesn't restrict

WHAT TO LOOK FOR:
- Virtual care services
- Telehealth consultations
- Telephone assessments/consultations
- Video assessments/consultations
- Any code that can be billed for a patient-facing virtual encounter

DO NOT INCLUDE:
- Physician-to-physician consultations (e-consults between doctors)
- In-person only codes
- Diagnostic procedures (ECG, imaging, labs)
- Codes you cannot find literally in the text

JSON only:
{{
  "section_key": "{section_key}",
  "found": true/false,
  "codes": [
    {{
      "code": "EXACT code (e.g., A003, K017)",
      "description": "COMPLETE description in sentence case",
      "fee": "EXACT fee value or units",
      "fee_type": "Standard|Surgeon|Assistant|Anaesthesia",
      "setting": "Hospital|Professional|N/A",
      "modality": "telephone|video|both",
      "page_found": <integer>,
      "level_3_heading": "subsection heading (e.g., GENERAL LISTINGS, INCISION)",
      "is_addon": true/false,
      "links_to": ["codes this links to"] or [],
      "condition": "conditions/notes if any",
      "reasoning": "brief explanation why this matches"
    }}
  ]
}}

IMPORTANT: For codes with multiple fee types (Surg/Asst/Anae) or settings (H/P), create SEPARATE entries for each combination.

If no telehealth/virtual codes in this section: {{"section_key": "{section_key}", "found": false, "codes": []}}"""

print(f"Alberta Code: {AB_CODE} - {AB_DESC} (${AB_FEE})")
print("Prompt builder ready")

## Cell 7: Phase 1 - Process Each Level 2 Section

In [ ]:
# Phase 1: Process each Level 2 section

prov_code = "ON"
prov_name = "Ontario"

print(f"{'='*70}")
print(f"PHASE 1: PROCESSING BY LEVEL 2 SECTIONS")
print(f"{'='*70}")

all_results = []
code_chunks = {}  # Store section text for Phase 2

def get_dynamic_max_tokens(char_count):
    """Set max_completion_tokens based on section size."""
    if char_count > 150000:
        return 20000
    elif char_count > 80000:
        return 14000
    elif char_count > 40000:
        return 10000
    elif char_count > 15000:
        return 6000
    else:
        return 4000

for section_key, section_info in tqdm(section_chunks.items(), desc="Processing sections"):
    
    char_count = section_info['char_count']
    max_tokens = get_dynamic_max_tokens(char_count)
    
    print(f"\n[{section_key[:60]}]")
    print(f"  Pages {section_info['start_page']}-{section_info['end_page']} | {char_count:,} chars | {max_tokens} max tokens")
    
    # Build prompt
    prompt = build_section_prompt(section_key, section_info)
    
    try:
        resp = client.chat.completions.create(
            model="gpt-5.1-2025-11-13",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_completion_tokens=max_tokens
        )
        track_cost(resp.usage.prompt_tokens, resp.usage.completion_tokens)
        
        content = resp.choices[0].message.content
        match = re.search(r'\{[\s\S]*\}', content)
        
        if match:
            result = json.loads(match.group())
            
            codes_found = result.get('codes', [])
            
            if result.get('found') and codes_found:
                print(f"  -> Found {len(codes_found)} code entries")
                
                for c in codes_found:
                    code = c.get('code', '')
                    fee = str(c.get('fee', ''))
                    fee_type = c.get('fee_type', 'Standard')
                    setting = c.get('setting', 'N/A')
                    modality = c.get('modality', '')
                    
                    # Unique key for dedup and Phase 2
                    unique_key = f"{code}_{fee}_{fee_type}_{setting}_{section_key}"
                    code_chunks[unique_key] = section_info['text']
                    
                    all_results.append({
                        'AB_Code': AB_CODE,
                        'AB_Description': AB_DESC,
                        'AB_Fee': AB_FEE,
                        'Target_Province': prov_code,
                        'Code': code,
                        'Description': c.get('description', ''),
                        'Fee': c.get('fee', ''),
                        'Fee_Type': fee_type,
                        'Setting': setting,
                        'Type': 'ADD-ON' if c.get('is_addon') else 'PRIMARY',
                        'Modality': modality,
                        'Links_To': ', '.join(c.get('links_to', [])) if c.get('links_to') else '',
                        'Condition': c.get('condition', ''),
                        'Reasoning': c.get('reasoning', ''),
                        'Level_1_Section': section_info['level_1'],
                        'Level_2_Subsection': section_info['level_2'],
                        'Level_3_Heading': c.get('level_3_heading', ''),
                        'Page_Found': c.get('page_found', ''),
                        '_unique_key': unique_key
                    })
            else:
                print(f"  -> No telehealth codes found")
        else:
            print(f"  -> ERROR: Could not parse JSON response")
            
    except Exception as e:
        print(f"  -> ERROR: {e}")

print(f"\n{'='*70}")
print(f"PHASE 1 COMPLETE")
print(f"{'='*70}")
print(f"Total code entries found: {len(all_results)}")
print(f"  - PRIMARY: {sum(1 for r in all_results if r['Type'] == 'PRIMARY')}")
print(f"  - ADD-ON: {sum(1 for r in all_results if r['Type'] == 'ADD-ON')}")
print(f"\nBy Fee Type:")
for ft in ['Standard', 'Surgeon', 'Assistant', 'Anaesthesia']:
    count = sum(1 for r in all_results if r['Fee_Type'] == ft)
    if count > 0:
        print(f"  - {ft}: {count}")
print(f"\nBy Setting:")
for s in ['Hospital', 'Professional', 'N/A']:
    count = sum(1 for r in all_results if r['Setting'] == s)
    if count > 0:
        print(f"  - {s}: {count}")
print(f"\nAPI calls: {total_calls} | Cost: ${total_cost:.2f}")

## Cell 8: Phase 1 Results Summary

In [ ]:
# Display Phase 1 results by section
import pandas as pd

df_phase1 = pd.DataFrame(all_results)

if len(df_phase1) > 0:
    print("PHASE 1 RESULTS BY SECTION")
    print("="*70)
    
    section_counts = df_phase1.groupby('Level_1_Section').size().sort_index()
    for section, count in section_counts.items():
        print(f"  {section[:50]:50} | {count:3} codes")
    
    print(f"\n{'='*70}")
    print("ALL CODES FOUND:")
    print("-"*70)
    for _, row in df_phase1.iterrows():
        print(f"  {row['Code']:8} | ${str(row['Fee']):>8} | {row['Fee_Type']:12} | {row['Setting']:12} | {row['Modality']:10} | pg {row['Page_Found']:>3}")
else:
    print("No codes found in Phase 1")

# Save Phase 1
phase1_file = 'phase1_on_section_pilot.xlsx'
df_phase1.to_excel(phase1_file, index=False)
print(f"\nSaved: {phase1_file}")

---
# Phase 2: Attribute Extraction
---

## Cell 9: Extract Rules of Application (General Preamble)

In [ ]:
import fitz  # PyMuPDF

# Extract General Preamble/Rules pages for Ontario (pages 1-126)
RULES_START_PAGE = 1
RULES_END_PAGE = 126

print(f"Extracting General Preamble/Rules (pages {RULES_START_PAGE}-{RULES_END_PAGE})...")

# Open source PDF
src_pdf = fitz.open(ON_PDF)

# Create new PDF with preamble pages
rules_pdf = fitz.open()
rules_pdf.insert_pdf(src_pdf, from_page=RULES_START_PAGE-1, to_page=RULES_END_PAGE-1)

# Save Rules PDF
rules_pdf_file = 'on_general_preamble.pdf'
rules_pdf.save(rules_pdf_file)
print(f"Saved: {rules_pdf_file} ({RULES_END_PAGE - RULES_START_PAGE + 1} pages)")

# Extract text from Rules for use in prompts
rules_of_application_text = ""
for page_num in range(RULES_START_PAGE - 1, RULES_END_PAGE):
    page = src_pdf[page_num]
    text = page.get_text()
    if text:
        rules_of_application_text += f"\n=== RULES PAGE {page_num + 1} ===\n{text}"

src_pdf.close()
rules_pdf.close()

print(f"Loaded General Preamble text: {len(rules_of_application_text):,} characters")

# Download the Rules PDF
files.download(rules_pdf_file)

## Cell 10: Load Extraction Taxonomy

In [ ]:
# Load extraction taxonomy
print("Upload extraction_taxonomy.xlsx:")
uploaded_tax = files.upload()

taxonomy_file = list(uploaded_tax.keys())[0]
df_taxonomy = pd.read_excel(taxonomy_file)

print(f"\nLoaded {len(df_taxonomy)} attributes:")
for _, row in df_taxonomy.iterrows():
    print(f"  - {row['attribute']}: {row['data_type']}")

# Build taxonomy reference string for prompts
taxonomy_reference = "\n".join([
    f"- {row['attribute']} ({row['data_type']}): {row['definition']} Taxonomy: {row['taxonomy']}"
    for _, row in df_taxonomy.iterrows()
])

print("\nTaxonomy loaded and ready for Phase 2")

## Cell 11: Phase 2 - Extract Attributes for Each Code

In [ ]:
# Phase 2: Extract attributes for each code using stored section chunks + rules

def build_attribute_prompt(code_info, chunk_text, rules_text, taxonomy_ref):
    """Build prompt to extract attributes for a single code."""
    return f"""You are a senior physician billing specialist extracting detailed attributes for an Ontario billing code.

CODE TO ANALYZE:
- Code: {code_info['Code']}
- Description: {code_info['Description']}
- Fee: {code_info['Fee']}
- Fee Type: {code_info['Fee_Type']}
- Setting: {code_info['Setting']}
- Type: {code_info['Type']}
- Section: {code_info.get('Level_1_Section', 'N/A')} > {code_info.get('Level_2_Subsection', 'N/A')}
- Condition (from Phase 1): {code_info.get('Condition', 'N/A')}

ATTRIBUTES TO EXTRACT:
{taxonomy_ref}

GENERAL PREAMBLE/RULES:
{rules_text[:50000]}

CODE-SPECIFIC SECTION:
{chunk_text[:30000]}

TASK:
Using ALL available information above, extract values for each attribute.

INSTRUCTIONS:
1. Use information from BOTH the General Preamble AND the code-specific section
2. For each attribute, extract the value if found, or null if not stated
3. For same_day_exclusions: return as array of code strings
4. For additional_notes: ONLY include important billing information not captured elsewhere

Return JSON only:
{{
  "modality": "telephone|video|both|in_person|asynchronous|null",
  "minimum_time_minutes": integer or null,
  "frequency_per_day": integer or null,
  "frequency_per_year": integer or null,
  "frequency_per_year_period": "annual|quarterly|90_days|monthly|null",
  "same_day_exclusions": ["code1", "code2"] or [] or null,
  "premium_extended_hours": "rate% code conditions" or null,
  "premium_location": "rate% code conditions" or null,
  "premium_age": "rate% conditions" or null,
  "premium_other": "rate% code conditions" or null,
  "additional_notes": "other important billing info" or null
}}"""

# Process each code
phase2_results = []

print(f"Extracting attributes for {len(all_results)} codes...")
print("="*70)

for idx, code_info in enumerate(tqdm(all_results, desc="Extracting attributes")):
    unique_key = code_info.get('_unique_key', '')
    chunk_text = code_chunks.get(unique_key, '')
    
    prompt = build_attribute_prompt(code_info, chunk_text, rules_of_application_text, taxonomy_reference)
    
    try:
        resp = client.chat.completions.create(
            model="gpt-5.1-2025-11-13",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_completion_tokens=1500
        )
        track_cost(resp.usage.prompt_tokens, resp.usage.completion_tokens)
        
        content = resp.choices[0].message.content
        match = re.search(r'\{[\s\S]*\}', content)
        
        if match:
            attrs = json.loads(match.group())
            
            # Convert same_day_exclusions array to string for Excel
            if attrs.get('same_day_exclusions') and isinstance(attrs['same_day_exclusions'], list):
                attrs['same_day_exclusions'] = ', '.join(attrs['same_day_exclusions'])
            
            phase2_results.append({
                '_unique_key': unique_key,
                **attrs
            })
            
            n_filled = sum(1 for v in attrs.values() if v is not None and v != 'null' and v != '')
            print(f"  {code_info['Code']} ({code_info['Fee_Type']}, {code_info['Setting']}): {n_filled} attributes")
        else:
            print(f"  {code_info['Code']}: No JSON found")
            phase2_results.append({'_unique_key': unique_key})
            
    except Exception as e:
        print(f"  {code_info['Code']}: Error - {e}")
        phase2_results.append({'_unique_key': unique_key})

print(f"\nPhase 2 complete: {len(phase2_results)} codes processed")
print(f"Total API cost: ${total_cost:.2f}")

## Cell 12: Combine Phase 1 + Phase 2 and Save

In [ ]:
# Combine Phase 1 and Phase 2 results

df_phase1 = pd.DataFrame(all_results)
df_phase2 = pd.DataFrame(phase2_results)

# Merge on _unique_key
df_combined = df_phase1.merge(df_phase2, on='_unique_key', how='left')

# Drop internal column
df_combined = df_combined.drop(columns=['_unique_key'])

# Reorder columns
column_order = [
    'AB_Code', 'AB_Description', 'AB_Fee', 'Target_Province',
    'Code', 'Description', 'Fee', 'Fee_Type', 'Setting', 'Type', 'Modality',
    'Links_To', 'Condition', 'Reasoning',
    'Level_1_Section', 'Level_2_Subsection', 'Level_3_Heading', 'Page_Found',
    'modality', 'minimum_time_minutes', 'frequency_per_day', 'frequency_per_year',
    'frequency_per_year_period', 'same_day_exclusions', 'premium_extended_hours',
    'premium_location', 'premium_age', 'premium_other', 'additional_notes'
]

final_columns = [c for c in column_order if c in df_combined.columns]
df_combined = df_combined[final_columns]

print(f"Combined DataFrame: {len(df_combined)} rows, {len(df_combined.columns)} columns")
df_combined

## Cell 13: Save Final Output

In [ ]:
# Save final results
print("="*70)
print("FINAL OUTPUT")
print("="*70)

output_file = 'on_section_pilot_complete.xlsx'
df_combined.to_excel(output_file, index=False)
print(f"\nSaved: {output_file}")
print(f"  - Rows: {len(df_combined)}")
print(f"  - Columns: {len(df_combined.columns)}")

# Summary
print(f"\n--- SUMMARY ---")
print(f"Total code entries: {len(df_combined)}")
print(f"  - PRIMARY: {len(df_combined[df_combined['Type'] == 'PRIMARY'])}")
print(f"  - ADD-ON: {len(df_combined[df_combined['Type'] == 'ADD-ON'])}")

print(f"\n--- BY FEE TYPE ---")
fee_type_counts = df_combined['Fee_Type'].value_counts()
for ft, count in fee_type_counts.items():
    print(f"  {ft}: {count}")

print(f"\n--- BY SETTING ---")
setting_counts = df_combined['Setting'].value_counts()
for s, count in setting_counts.items():
    print(f"  {s}: {count}")

print(f"\n--- BY SECTION ---")
section_counts = df_combined.groupby('Level_1_Section').size().sort_index()
for section, count in section_counts.items():
    print(f"  {section[:50]:50} | {count:3}")

print(f"\n--- COST ---")
print(f"Total API calls: {total_calls}")
print(f"Total cost: ${total_cost:.2f}")

# Download
files.download(output_file)